In [3]:
import pandas as pd
import geopandas as gpd

In [59]:
# Residential Land Use Codes from MA Dept of Revenue
# https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf
# Codes are 101*-109*, 031*, and 013*
# Often include suffixes (letters, zeroes or no character), thus regex *?
USE_CODES = '^1[0-1][1-2]*?|^013*?|^031*?'
def read_res(file_list, uses = USE_CODES):
    df = pd.DataFrame()
    for file in file_list:
        df = df.append(gpd.read_file(file), ignore_index=True)
    df = df[[
        'PROP_ID',
        'OWNER1',
        'OWN_ADDR',
        'OWN_CITY',
        'OWN_STATE',
        'OWN_ZIP',
        'OWN_CO',
        'USE_CODE',
        'CITY',
        'FY']]
    df = df.rename({
        'PROP_ID': 'pid',
        'OWNER1': 'own_name', 
        'OWN_ADDR': 'own_add', 
        'OWN_CITY': 'own_city', 
        'OWN_STATE': 'own_state',
        'OWN_ZIP': 'own_zip', 
        'OWN_CO': 'own_country',
        'CITY': 'city',
        'USE_CODE': 'use',
        'FY': 'year'}, 
        axis='columns')
    df = df[df['use'].str.contains(uses, regex=True)]
    return df

In [85]:
# Data from MassGIS Standardized Assessor's Parcels
# https://docs.digital.mass.gov/dataset/massgis-data-standardized-assessors-parcels
# Medford, Cambridge, and Somerville all last updated FY 2019
files = ['som_assess.dbf', 'cam_assess.dbf', 'med_assess.dbf']
df = read_res(files)

In [86]:
import re
from unidecode import unidecode
replace_list = ['FAMILY', 'IRREVOCABLE', 'NOMINEE', 'REVOCABLE', 
                'REALTY', 'REAL ESTATE', 'TRUSTEES OF', 'TRUSTEE OF', 
                'TRUSTEE', 'TRST', 'TRUST', 'LTD',
                'LLC', 'HOLDINGS', 'REALTORS', 'LIMITED PARTNERSHIP', 
                'FOR LIFE', 'ESTATE OF', 'ESTATE', 'TR.']
def clean(c):
    c = c.replace('|'.join(map(re.escape, replace_list)), '', regex=True)
    return c

df['own_name_clean'] = clean(df['own_name'])
df.head()

,pid,own_name,own_add,own_city,own_state,own_zip,own_country,use,city,year,own_name_clean
16,102_A_1,DEIRMENJIAN REAL ESTATE LLC,6 WINCHESTER ST,MEDFORD,MA,02155,None,1110,SOMERVILLE,2019,DEIRMENJIAN
17,102_A_2,FALCETANO WILLIAM TRUSTEE,337 SOMERVILLE AVE 2ND FLOOR,SOMERVILLE,MA,02143,USA,1110,SOMERVILLE,2019,FALCETANO WILLIAM
18,102_A_3,GALLUZZO ELLEN FOR LIFE,9 AUSTIN ST,SOMERVILLE,MA,02145,USA,1040,SOMERVILLE,2019,GALLUZZO ELLEN
19,102_A_4,SOUSA JOSE C & MATILDE,23 BENEDICT ST,SOMERVILLE,MA,02145,USA,1050,SOMERVILLE,2019,SOUSA JOSE C & MATILDE
20,102_B_1,VALENCIA MILAGRO TRUSTEE,18 BENEDICT ST,SOMERVILLE,MA,02145,None,1050,SOMERVILLE,2019,VALENCIA MILAGRO


In [72]:
import dedupe

# settings_file = 'csv_example_learned_settings'
# training_file = 'csv_example_training.json'

fields = [
    {'field': 'own_name_clean', 'type': 'String'},
#     {'field': 'Phone', 'type': 'String', 'has missing': True},
    ]

df_dict = df.to_dict('index')

deduper = dedupe.Dedupe(fields)

deduper.prepare_training(df_dict)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (firstTokenPredicate, own_name_clean), SimplePredicate: (twoGramFingerprint, own_name_clean))


In [73]:
dedupe.console_label(deduper)

own_name_clean : NORCROSS BOYD A & MARY J EE

own_name_clean : NORCROSS BOYD A & MARY J E

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


own_name_clean : LENNON CATHERINE EE

own_name_clean : LENNON CATHERINE 

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, own_name_clean), SimplePredicate: (sortedAcronym, own_name_clean))
own_name_clean : TOURO INVESTMENTS #9 

own_name_clean : TOURO INVESTMENTS 

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

own_name_clean : CHEN, LINAN

own_name_clean : CHEN WILLIAM K & HENRY G

19/10 positive, 49/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


own_name_clean : THE 60 FOUNTAIN STREET  

own_name_clean : THE SUN 1 

19/10 positive, 50/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [74]:
deduper.train()
with open('csv_example_training.json', 'w') as tf:
    deduper.write_training(tf)
with open('settings', 'wb') as sf:
    deduper.write_settings(sf)

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.001000, score 0.763677932268224
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, own_name_clean), SimplePredicate: (tokenFieldPredicate, own_name_clean))


In [75]:
print('Clustering...')
clustered_dupes = deduper.partition(df_dict, 0.5)
print('# duplicate sets', len(clustered_dupes))

Clustering...


INFO:dedupe.blocking:10000, 0.1782162 seconds
INFO:dedupe.blocking:20000, 0.3309222 seconds
INFO:dedupe.blocking:30000, 0.4865912 seconds
INFO:dedupe.blocking:40000, 0.6323362 seconds
INFO:dedupe.blocking:50000, 0.7865862 seconds


# duplicate sets 47133


In [87]:
rid = []
clst = []
conf = []
count = []
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        count.append(len(records))
        rid.append(record_id)
        clst.append(cluster_id)
        conf.append(score)
        
clust = pd.DataFrame(list(zip(clst, conf, count)), 
                  columns =['clst', 'conf', 'count'],
                  index = rid
                 )
df = df.join(clust)

In [82]:
df[df['own_name']  5]

,pid,own_name,own_add,own_city,own_state,own_zip,own_country,use,city,year,own_name_clean,clst,conf,count
149,103_C_20,VENEZIANO JAMES M,PO BOX 556,SOMERVILLE,MA,02143,USA,1050,SOMERVILLE,2019,VENEZIANO JAMES M,159,0.977959,9
150,103_C_21,VENEZIANO JAMES M TRUSTEE,PO BOX 556,SOMERVILLE,MA,02143,USA,1110,SOMERVILLE,2019,VENEZIANO JAMES M,159,0.980873,9
151,103_C_22,VENEZIANO JAMES TRUSTEE,PO BOX 556,SOMERVILLE,MA,02143,USA,1050,SOMERVILLE,2019,VENEZIANO JAMES,159,0.976838,9
158,103_C_26,VENEZIANO JAMES M TRUSTEE,PO BOX 556,SOMERVILLE,MA,02143,USA,1110,SOMERVILLE,2019,VENEZIANO JAMES M,159,0.980873,9
234,103_G_13,DIRENZO R & IOLANDA & MCKENZIE D TRUSTEE,17 ROYALL ST,MEDFORD,MA,02155,None,1110,SOMERVILLE,2019,DIRENZO R & IOLANDA & MCKENZIE D,242,0.982204,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63975,X-10_128,BARROS JOSE D & MARIA,252 HIGH ST,MEDFORD,MA,02155,USA,1040,MEDFORD,2019,BARROS JOSE D & MARIA,197,0.687651,6
64070,X-10_96,SANTOS MANUEL C,20 HIGGINS AVE,MEDFORD,MA,02155,USA,1040,MEDFORD,2019,SANTOS MANUEL C,42,0.915227,6
64272,Y-10_23,LACOURT FOUNDATION LLC,30 COLLEGE AVE,SOMERVILLE,MA,02144,USA,1040,MEDFORD,2019,LACOURT FOUNDATION,1254,0.976714,20
64324,Y-10_59,MAHER DAVID F,962 BROADWAY,SOMERVILLE,MA,02144,USA,1040,MEDFORD,2019,MAHER DAVID F,1202,0.825011,9


In [91]:
df['pid'] = df['pid'].replace('_', '-', regex=True)
df.set_index('pid')
df.head()
df.to_csv('parcels_clustered.csv')